In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math

AquiferDoganella=pd.read_csv('Aquifer_Doganella.csv')
AquiferAuser=pd.read_csv('Aquifer_Auser.csv')
WaterSpringAmiata=pd.read_csv('Water_Spring_Amiata.csv')
LakeBilancino=pd.read_csv('Lake_Bilancino.csv')
WaterSpringMadonna=pd.read_csv('Water_Spring_Madonna_di_Canneto.csv')
AquiferLuco=pd.read_csv('Aquifer_Luco.csv')
AquiferPetrignano=pd.read_csv('Aquifer_Petrignano.csv')
WaterSpringLupa=pd.read_csv('Water_Spring_Lupa.csv')
RiverArno=pd.read_csv('River_Arno.csv')

# juntando todos os dataset em um (join all dataset in just one called 'dados')
dados = [AquiferDoganella,AquiferAuser,WaterSpringAmiata,LakeBilancino,WaterSpringMadonna,
           AquiferLuco,AquiferPetrignano,WaterSpringLupa,RiverArno]
dados_arquivos=['Aquifer_Doganella.csv', 'Aquifer_Auser.csv', 'Water_Spring_Amiata.csv', 'Lake_Bilancino.csv', 'Water_Spring_Madonna_di_Canneto.csv', 'Aquifer_Luco.csv', 'Aquifer_Petrignano.csv', 'Water_Spring_Lupa.csv', 'River_Arno.csv']

def plot1(inputdata,features,target_var,ylabel1,ylabel2):
    fig, ax1= plt.subplots(figsize=(15,5))
    ax1.bar(inputdata['Year-mon'],inputdata[features])
    ax1.spines['left'].set_color('black')
    ax1.spines['left'].set_linewidth(3)
    ax1.legend([features],loc=2)
    ax2 =ax1.twinx()
    ax2.plot(inputdata['Year-mon'],inputdata[target_var],color='grey')
    ax2.spines['right'].set_color('blue')
    ax2.spines['right'].set_linewidth(3)
    ax1.set_ylabel(ylabel1)
    ax1.set_xticks(range(0,len(inputdata),10))
    ax1.set_xticklabels(inputdata['Year-mon'][range(0,len(inputdata),10)],rotation=90)
    ax2.set_ylabel(ylabel2)
    ax2.set_xticks(range(0,len(inputdata),10))
    ax2.set_xticklabels(inputdata['Year-mon'][range(0,len(inputdata),10)],rotation=90)
    ax2.legend([target_var[0]],loc=1)
    
# converting dates
for i in range(len(dados)): 
  dados[i].drop(dados[i][dados[i].Date.isnull()].index,inplace = True,axis=0)
  dados[i]['Year-mon']=pd.to_datetime(dados[i].Date).apply(lambda x: x.strftime('%Y-%m'))
dados_monthly = [dados[i].groupby('Year-mon').mean().reset_index() for i in range(len(dados))]

dadosSelect=['Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2',
       'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
       'Depth_to_Groundwater_Pozzo_5', 'Depth_to_Groundwater_Pozzo_6',
       'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8',
       'Depth_to_Groundwater_Pozzo_9', ]
plot1(dados_monthly[0],'Rainfall_Monteporzio',dadosSelect,'Rainfall, mm','Groundwater Level, m')

# setup a variavel aim
aim = [[col for col in dados_monthly[0].columns if 'Depth' in col]]
aim.append([col for col in dados_monthly[1].columns if 'Depth' in col])
aim.append([col for col in dados_monthly[2].columns if 'Flow_Rate' in col])
aim.append([col for col in dados_monthly[3].columns if 'Lake' in col])
aim.append([col for col in dados_monthly[4].columns if 'Flow_Rate' in col])
aim.append([col for col in dados_monthly[5].columns if 'Depth' in col])
aim.append([col for col in dados_monthly[6].columns if 'Depth' in col])
aim.append([col for col in dados_monthly[7].columns if 'Flow_Rate' in col])
aim.append([col for col in dados_monthly[8].columns if 'Hydrometry' in col])

for i in range(len(dados_monthly)):
    dados_monthly[i].loc[dados_monthly[i]['Year-mon'].str.split('-').str[1].str.contains('12|01|02'),'season']=1
    dados_monthly[i].loc[dados_monthly[i]['Year-mon'].str.split('-').str[1].str.contains('03|04|05'),'season']=2
    dados_monthly[i].loc[dados_monthly[i]['Year-mon'].str.split('-').str[1].str.contains('06|07|08'),'season']=3
    dados_monthly[i].loc[dados_monthly[i]['Year-mon'].str.split('-').str[1].str.contains('09|10|11'),'season']=4
    
    
for i in range(len(dados_monthly)):
    dados_monthly[i].loc[dados_monthly[i].iloc[:,1]>(dados_monthly[i].iloc[:,1].mean()+dados_monthly[i].iloc[:,1].std()),'dryness']=4
    dados_monthly[i].loc[(dados_monthly[i].iloc[:,1]<=(dados_monthly[i].iloc[:,1].mean()+dados_monthly[i].iloc[:,1].std())) & (dados_monthly[i].iloc[:,1]>dados_monthly[i].iloc[:,1].mean()),'dryness']=3
    dados_monthly[i].loc[(dados_monthly[i].iloc[:,1]>=(dados_monthly[i].iloc[:,1].mean()-dados_monthly[i].iloc[:,1].std())) & (dados_monthly[i].iloc[:,1]<=dados_monthly[i].iloc[:,1].mean()),'dryness']=2
    dados_monthly[i].loc[dados_monthly[i].iloc[:,1]<(dados_monthly[i].iloc[:,1].mean()-dados_monthly[i].iloc[:,1].std()),'dryness']=1

# create features of previous month for analyzes 
for i in range(len(dados_monthly)):
    newDado =[newDado for newDado in dados_monthly[i].columns if newDado not in aim[i]][1:]
    for j in range(len(newDado)):
        dados_monthly[i][f'{newDado[j]}_previous_month1']=np.append([np.nan],dados_monthly[i].iloc[:-1][f'{newDado[j]}'].values)

        #Accuracy

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
f,ax3 = plt.subplots(9,2,figsize=(10,50))
# dataframe
index=[dados_arquivos[j][:-4]+': '+item for j, sublist in enumerate(aim) for item in sublist]
start = pd.DataFrame(columns=['MSE','MAE','R-Squared','Top 3 important features'],index=index)
for j in range(len(aim)):
    for i,target in enumerate(aim[j]):
        #select non-null values
        idx1 = dados_monthly[j][target].notnull()
        data = dados_monthly[j][idx1].fillna(dados_monthly[j][idx1].mean())
        x = data.drop(aim[j],axis=1)
        x = x.drop('Year-mon',axis=1)
        y = data[target]
        x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3)
        np.random.seed(1234)
        RFG = RandomForestRegressor(n_estimators=40,random_state=0)
        RFG.fit(x_train,y_train)
        importance = RFG.feature_importances_
        

        # output statistics
        start.loc[f'{dados_arquivos[j][:-4]}: {target}','MSE']=f'{mean_squared_error(y_test,RFG.predict(x_test)):.3f}'
        start.loc[f'{dados_arquivos[j][:-4]}: {target}','MAE']=f'{mean_absolute_error(y_test,RFG.predict(x_test)):.3f}' 
        start.loc[f'{dados_arquivos[j][:-4]}: {target}','R-Squared']=f'{r2_score(y_test,RFG.predict(x_test)):.3f}'
        start.loc[f'{dados_arquivos[j][:-4]}: {target}','Top 3 important features']= [list(x.columns[np.argsort(importance)[::-1][:3]])]
        
        # an example of prediction in visulization
        if j == 0:
            ax3[i,0].plot(y_test,RFG.predict(x_test),'.')
            ax3[i,0].plot(np.linspace(np.amin(y_test),np.amax(y_test),100),np.linspace(np.amin(y_test),np.amax(y_test),100),'k')
            ax3[i,0].set_ylabel('Predicted')
            ax3[i,0].set_xlabel('Observed')
            ax3[i,0].text(np.min(y_test),np.max(y_test)-1,f'MSE: {mean_squared_error(y_test,RFG.predict(x_test)):.3f}')
            ax3[i,0].text(np.min(y_test),np.max(y_test)-2,f'MAE: {mean_absolute_error(y_test,RFG.predict(x_test)):.3f}')
            ax3[i,0].text(np.min(y_test),np.max(y_test)-3,f'R-Squared: {r2_score(y_test,RFG.predict(x_test)):.3f}' )
            ax3[i,0].set_title(f'{target}, meter')   
            ax3[i,1].bar(range(len(importance)),importance)
            ax3[i,1].set_xticks(range(len(importance)))
            ax3[i,1].set_xticklabels(x.columns,rotation=90)
            ax3[i,1].set_ylabel('Importance')
    plt.tight_layout()